In [1]:
import requests as req
import json as js
from dtos.dtos import *

In [2]:
baseUrl:str = "http://localhost:3010/maas"
baseHeader = {"user" : "{'_id' : '60f527b78c90cf6b8bf450eb', 'client_id' : '60f1b8e0203c9b20acb9ad26'}", 
              "Content-Type" : "application/json"}
main_date = "2024-05-28T15:00:36.459Z"
provider_id = "0b13f2d6-d5cb-48aa-aeed-ed3a5a7f1da6"

<h1> Check the auth of Cooltra <h1>

In [3]:
dataAuth = """
    {
     "grant_type": "client_credentials",
     "client_id": "JTrLV2ByJYJ5w1FU0M8tjkrnOh2U8uD",
     "client_secret": "zEoTATE6iciep+t8iHfbLmmTbJe3k54C2ZpCavXTXe/ZO0FJqbhUsM0prGv/JwgQ",
     "scope": "offers"
    }
           """

# authorize
#########################################################################
cooltraBaseUrl = "https://maas.zeus-stage.cooltra.com"

response = req.request("POST",
                       url=cooltraBaseUrl + "/oauth/token",
                       headers= {"Content-Type" : "application/json", "Accept" : "application/json"},
                       data=dataAuth
                       )

result : CooltraAuthResponseDTO = CooltraAuthResponseDTOSchema().loads(response.text)
token = "Bearer " + result.access_token

#########################################################################


In [20]:
token

'Bearer eyJhbGciOiJSUzI1NiIsImtpZCI6ImRmOGIxNTFiY2Q5MGQ1YjMwMjBlNTNhMzYyZTRiMzA3NTYzMzdhNjEiLCJ0eXAiOiJKV1QifQ.eyJwcm92aWRlcl9pZCI6Ik1PVkVTSU9OIiwiaXNzIjoiaHR0cHM6Ly9zZWN1cmV0b2tlbi5nb29nbGUuY29tL2Nvb2x0cmEtc3RhZ2UiLCJhdWQiOiJjb29sdHJhLXN0YWdlIiwiYXV0aF90aW1lIjoxNzE4NjE0NTc0LCJ1c2VyX2lkIjoiZXJMVlVwN21IcGI3S1NLdXk3d1hBaG93NU1iMiIsInN1YiI6ImVyTFZVcDdtSHBiN0tTS3V5N3dYQWhvdzVNYjIiLCJpYXQiOjE3MTg2MTQ1NzQsImV4cCI6MTcxODYxODE3NCwiZW1haWwiOiJqdHJsdjJieWp5ajV3MWZ1MG04dGprcm5vaDJ1OHVkQGNvb2x0cmEuY29tIiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJmaXJlYmFzZSI6eyJpZGVudGl0aWVzIjp7ImVtYWlsIjpbImp0cmx2MmJ5anlqNXcxZnUwbTh0amtybm9oMnU4dWRAY29vbHRyYS5jb20iXX0sInNpZ25faW5fcHJvdmlkZXIiOiJwYXNzd29yZCJ9fQ.n3wlvi4rOS5ZkfjZeC2FfB4fJUcJBZDsW0BGhWYGHdqOv_HxjFjluzRyxqZ_q4rFHwqvM5GtbnDiwAbeEeZCfpos38uk2hT0RUGT04zCUGMk8K471S1WTusRtAty_asDyB7Rlci3rWqbvcrcgAQXaf6idXlBeBHWv9Zrb1WbFkX7CqvimKxtD4rx4pkNZUc4ymygJZ7cCzTeP1GARt8oTgOTZ5Zy38Dw2C7UwN4mdv9tUM49kmhW9IcbLqPpQ6v40sz-HmPVhh1506PectHNmY8DuUh_E0igI24sk-Vo7JSFIWrz0xNVSCdB3SEORk

<h1> User registration </h1>

In [21]:
newUser = CooltraUserRegistrationRequestDTO(
    external_user_id="f468fdd1-c6f3-4dca-9dc3-dbcba6b8ea8c",
    first_name="John",
    middle_name="Doe",
    last_name="Doe",
    email="test@test.com",
    phone="+393493494949",
    language="en",
    gender="m",
    birthdate="1990-01-01",
    address_street="123 Main St",
    address_city="New York",
    address_zip="12345",
    address_country_code="US",
    address_state="NY",
    provider="erLVUp7mHpb7KSKuy7wXAhow5Mb2",
    provider_code="cooltra"
)


response = req.request("POST", 
            baseUrl + "/moto-rental/register",
            headers=baseHeader,
            data=CooltraUserRegistrationRequestDTOSchema().dumps(newUser)
            )



In [12]:
response.text

'{"errors":["Cooltra Error : {\\u0022phone\\u0022:[\\u0022is_invalid\\u0022]}"],"code":"cooltra__data_invalid"}'

<h1> Get configuration </h1>

In [5]:
response = req.request("GET", 
                       baseUrl + "/moto-rental/configuration", 
                       headers=baseHeader,
                       params={"providerCode":"cooltra"})
result : CooltraConfigurationResponseDTO = CooltraConfigurationResponseDTOSchema().loads(response.text)
all_ids = [system.id for system in result.systems]

In [6]:
all_ids

['rome',
 'madrid',
 'paris',
 'berlin',
 'zurich',
 'valencia',
 'seville',
 'turin',
 'lisbon',
 'barcelona',
 'milan']

<h1> Get vehicles in specific zone </h1>

In [7]:
response = req.request("GET", 
                       baseUrl + "/moto-rental/vehicles", 
                       headers=baseHeader, 
                       params={"providerCode":"cooltra", "systemId" : "barcelona"})
vehicles:List[CooltraMotoRentalVehicleDTO] = CooltraMotoRentalVehicleDTOSchema().loads(response.text, many=True)
first_vehicle_id = vehicles[0].id

<h1> Get specific vehicle </h1>

In [15]:
response = req.request("GET", 
                       baseUrl + f"/moto-rental/vehicles/{first_vehicle_id}", 
                       headers=baseHeader, 
                       params={"providerCode":"cooltra", "systemId" : "rome"})
result : CooltraMotoRentalVehicleDTO = CooltraMotoRentalVehicleDTOSchema().loads(response.text)

In [16]:
result

CooltraMotoRentalVehicleDTO(id=15751883946, license_plate='FAKE-H5648UAN', position=[2.233801729409702, 41.4638017294097], range=50, battery_percentage=72, model_id=2)

<h1> Reserve vehicle </h1>

In [23]:
reservationBody = CooltraReservationRequestDTO(
    vehicle_id = first_vehicle_id,
    provider_code="cooltra"
)

response = req.request("POST", 
                       baseUrl + f"/moto-rental/reserve", 
                       headers=baseHeader, 
                       data=CooltraReservationRequestDTOSchema().dumps(reservationBody))

result : CooltraReservationInfoDTOSchema = CooltraReservationInfoDTOSchema().loads(response.text)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [20]:
response.text

'{"id":null,"system_id":null,"created_at":"0001-01-01T00:00:00","expires_at":"0001-01-01T00:00:00","expires_in":0,"vehicle":null}'

In [44]:
response.ok

<h1> Check in to vehicle </h1>

In [9]:
reservationBody = CooltraCheckInRequestDTO(
    provider_id = "cooltra"
)

response = req.request("POST", 
                       baseUrl + f"/moto-rental/check-in", 
                       headers=baseHeader, 
                       data=CooltraCheckInRequestDTOSchema().dumps(reservationBody))

result : CooltraCheckInInfoDTO = CooltraCheckInInfoDTOSchema().loads(response.text)

ValidationError: {'id': ['Missing data for required field.'], 'system_id': ['Missing data for required field.'], 'reservation_id': ['Missing data for required field.'], 'started_at': ['Missing data for required field.'], 'duration': ['Missing data for required field.'], 'state': ['Missing data for required field.'], 'vehicle': ['Missing data for required field.'], 'errors': ['Unknown field.'], 'code': ['Unknown field.']}

In [10]:
response.text

'{"errors":["ProviderCode: The ProviderCode field is required."],"code":"validation_failed"}'